### wandb quickstart

In [1]:
import wandb
import random

wandb.login(key='46c3b8e339b3fb22dc286204510c8af5b2c3e2e5')

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="test_still_work_in_russia",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/bspanfilov/.netrc
wandb: Currently logged in as: bspanfilov (tmboris). Use `wandb login --relogin` to force relogin


acc,▁▃▇▆▆█▇█
loss,█▄▆▂▁▃▁▁
acc,0.88354
loss,0.11338


### shape reaserch

In [2]:
import pickle

with open('/Users/bspanfilov/Documents/edu/hse-4/Звук/hw1_asr/pytorch_project_template/src/data.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

In [3]:
type(loaded_data)

list

In [5]:
loaded_data[0]

{'audio': tensor([[0.0006, 0.0002, 0.0002,  ..., 0.0004, 0.0005, 0.0006]]),
 'spectrogram': tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [3.1197e-04, 1.1608e-03, 1.2813e-03,  ..., 2.6680e-04,
           1.4664e-03, 9.7162e-04],
          [1.6797e-03, 6.2502e-03, 6.8988e-03,  ..., 1.4365e-03,
           7.8957e-03, 5.2315e-03],
          ...,
          [3.9049e-06, 2.1788e-05, 4.9069e-06,  ..., 9.1084e-06,
           6.2222e-06, 7.5193e-06],
          [7.5832e-06, 1.1364e-05, 7.0826e-06,  ..., 3.4638e-06,
           6.5170e-06, 8.8364e-06],
          [9.0198e-07, 5.7118e-06, 1.2817e-05,  ..., 6.4256e-06,
           2.8519e-06, 4.0951e-06]]]),
 'text': "hugh's written a delightful part for her and she's quite inexpressible",
 'text_encoded': tensor([[ 8., 21.,  7.,  8., 19., 27., 23., 18.,  9., 20., 20.,  5., 14., 27.,
           1., 27.,  4.,  5., 12.,  9.,  7.,  8., 20.,  6., 21., 12., 27., 16.,
           1., 18., 20., 2

In [6]:
len(loaded_data)

2

In [9]:
len(loaded_data[0]['text_encoded'][0])

68

In [48]:
loaded_data[1]['text_encoded'].squeeze(0).shape


torch.Size([172])

In [46]:
texts = [loaded_data[i]['text_encoded'][0] for i in range(len(loaded_data))]

In [12]:
len(texts)

2

In [13]:
from torch.nn.utils.rnn import pad_sequence

padded = pad_sequence(texts)

In [19]:
type(padded)

torch.Tensor

In [20]:
padded.shape

torch.Size([172, 2])

In [22]:
loaded_data[0]['spectrogram'].shape

torch.Size([1, 128, 319])

In [52]:
loaded_data[1]['spectrogram'].squeeze(0).permute(1, 0).shape

torch.Size([1165, 128])

In [54]:
import torch

pad_sequence([sample['spectrogram'].squeeze(0).permute(1, 0) for sample in loaded_data], batch_first=True).permute(0, 2, 1).shape


torch.Size([2, 128, 1165])

In [56]:
import math

In [58]:
rnn_input_size = 128
rnn_input_size = int(math.floor(rnn_input_size + 2 * 20 - 41) / 2 + 1)
rnn_input_size = int(math.floor(rnn_input_size + 2 * 10 - 21) / 2 + 1)
rnn_input_size *= 32
rnn_input_size

1024

$$
\text{HardTanh}(x) = \begin{cases}
            \text{max\_val} & \text{ if } x > \text{ max\_val } \\
            \text{min\_val} & \text{ if } x < \text{ min\_val } \\
            x & \text{ otherwise } \\
        \end{cases}
        $$

### cer-wer debug

In [4]:
import editdistance
def calc_cer(target_text, predicted_text) -> float:
    if len(target_text) == 0:
        return 1
    return editdistance.eval(target_text.split(), predicted_text.split()) / len(target_text.split())


def calc_wer(target_text, predicted_text) -> float:
    if len(target_text) == 0:
        return 1
    return editdistance.eval(target_text, predicted_text) / len(target_text)


In [5]:
target_text = 'a a a'
predicted_text = 'a b a'

In [6]:
calc_cer(
    target_text=target_text,
    predicted_text=predicted_text
)

0.3333333333333333

In [7]:
calc_wer(
    target_text=target_text,
    predicted_text=predicted_text
)

0.2

### binarizing lm

In [1]:
!git clone https://github.com/kpu/kenlm.git

Cloning into 'kenlm'...
remote: Enumerating objects: 14170, done.
remote: Counting objects: 100% (483/483), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 14170 (delta 167), reused 410 (delta 132), pack-reused 13687 (from 1)
Receiving objects: 100% (14170/14170), 5.91 MiB | 2.69 MiB/s, done.
Resolving deltas: 100% (8047/8047), done.


In [4]:
%cd kenlm/

/Users/bspanfilov/Documents/edu/hse-4/Звук/hw1_asr/pytorch_project_template/src/kenlm


In [5]:
!mkdir -p build
%cd build

/Users/bspanfilov/Documents/edu/hse-4/Звук/hw1_asr/pytorch_project_template/src/kenlm/build


In [13]:
!cmake -DCMAKE_CXX_STANDARD=14 ..
!make -j4

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- Could NOT find Eigen3 (missing: Eigen3_DIR)
CMake Warning (dev) at CMakeLists.txt:97 (find_package):
  Policy CMP0167 is not set: The FindBoost module is removed.  Run "cmake
  --help-policy CMP0167" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Configuring done (0.1s)
-- Generating done (0.2s)
-- Build files have been written to: /Users/bspanfilov/Documents/edu/hse-4/Звук/hw1_asr/pytorch_project_template/src/kenlm/build
[  1%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum-dtoa.cc.o
[  3%] Building CXX object util/CMakeFiles/ken

In [14]:
!bin/build_binary /Users/bspanfilov/Documents/edu/hse-4/Звук/hw1_asr/3-gram.pruned.1e-7.arpa /Users/bspanfilov/Documents/edu/hse-4/Звук/hw1_asr/3-gram.pruned.1e-7.bin

Reading /Users/bspanfilov/Documents/edu/hse-4/Звук/hw1_asr/3-gram.pruned.1e-7.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


### smth new

In [2]:
'^'.upper()

'^'

In [4]:
class a:
    def __init__(self, b):
        self.b = b
    def __call__(self):
        raise NotImplementedError()

class A(a):
    def __call__(self):
        self.b += 1

In [6]:
t = a(5)

In [7]:
t()

NotImplementedError: 